In [408]:
import pandas as pd
import datetime
import re
import matplotlib.pyplot as plt

In [409]:
data = pd.read_csv('C:\\Users\\RAVID\\Desktop\\Ariel\\3rd year\\semester 2\\כרייה וניתוח נתונים מתקדם בפייתון\\מטלות\\מטלה 2\\matala2_cosmetics_2019-Nov.csv')
data.info()

C:\Users\RAVID\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4635837 entries, 0 to 4635836
Data columns (total 9 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   event_time     object
 1   event_type     object
 2   product_id     int64 
 3   category_id    int64 
 4   category_code  object
 5   brand          object
 6   price          object
 7   user_id        int64 
 8   user_session   object
dtypes: int64(3), object(6)
memory usage: 318.3+ MB


**סעיף 1**

In [410]:
#ננקה את הדאטה מערכים חסרים
dataS = data.sort_values(by=['user_session', 'event_time'])
dataS = data[(data['user_session'].notnull())]
dataS = dataS.reset_index(drop=True)
dataS

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:02 UTC,view,5802432,1487580009286598681,NaN,NaN,0.32,562076640,09fafd6c-6c99-46b1-834f-33527f4de241
1,2019-11-01 00:00:09 UTC,cart,5844397,1487580006317032337,NaN,NaN,2.38,553329724,2067216c-31b5-455d-a1cc-af0575a34ffb
2,2019-11-01 00:00:10 UTC,view,5837166,1783999064103190764,NaN,pnb,22.22,556138645,57ed222e-a54a-4907-9944-5a875c2d7f4f
3,2019-11-01 00:00:11 UTC,cart,5876812,1487580010100293687,NaN,jessnail,3.16,564506666,186c1951-8052-4b37-adce-dd9644b1d5f7
4,2019-11-01 00:00:24 UTC,remove_from_cart,5826182,1487580007483048900,NaN,NaN,3.33,553329724,2067216c-31b5-455d-a1cc-af0575a34ffb
...,...,...,...,...,...,...,...,...,...
4635019,2019-11-30 23:59:46 UTC,view,5830317,1487580009496313889,NaN,NaN,4.76,457678989,ee50b160-a4db-4722-8751-6812c5b38295
4635020,2019-11-30 23:59:47 UTC,view,5733064,1487580004832248652,NaN,beautix,9.37,422196217,ab5e6dd5-8700-4ecc-a300-9f1eca5d1a95
4635021,2019-11-30 23:59:47 UTC,view,5867785,1487580007835370453,NaN,kims,31.1,572579084,d42865b7-7e04-4038-9be0-a59165625f06
4635022,2019-11-30 23:59:57 UTC,view,5779406,2151191071051219817,NaN,NaN,2.86,540006764,d4b5aa49-d731-40f1-92f1-277416d6e063


In [411]:
#נשנה את הטייפ של העמודה כדי שנוכל לבצע בה חיסור
dataS['event_time'] = dataS['event_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S UTC'))
dataS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4635024 entries, 0 to 4635023
Data columns (total 9 columns):
 #   Column         Dtype         
---  ------         -----         
 0   event_time     datetime64[ns]
 1   event_type     object        
 2   product_id     int64         
 3   category_id    int64         
 4   category_code  object        
 5   brand          object        
 6   price          object        
 7   user_id        int64         
 8   user_session   object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 318.3+ MB


In [ ]:
dataS['duration_to_next_event'] = dataS.groupby('user_session')['event_time'].apply(lambda x: x.shift(-1) - x).fillna(pd.Timedelta(seconds=0))
dataS['duration_to_next_event']= dataS['duration_to_next_event'].apply(lambda x: x.total_seconds())

dataS

**סעיף 2**

In [ ]:
dataS['MoreThan'] = dataS.groupby('user_id')['event_time'].apply(lambda x: (x - x.shift(1)) > datetime.timedelta(days=5)).reset_index(drop=True).astype(int)
dataS['funnel_number'] = dataS.groupby('user_id')['MoreThan'].cumsum()
dataS['funnel_number'] = dataS['funnel_number'].fillna(0) + 1
dataS = dataS.drop('MoreThan', axis=1)
dataS

**סעיף 3**

In [ ]:
dataS['one'] = 1
dataS['index_in_funnel'] = dataS.groupby(['user_id','funnel_number','user_session'])['one'].cumsum()
dataS = dataS.drop('one', axis=1)
dataS

In [ ]:
dataS[dataS['user_session'] == '11f77a13-3365-4014-8d60-bbb933b8c3f1']

**סעיף 4**

In [ ]:
dataS['priceNew']= dataS['price'].apply(lambda x: re.findall('([0-9]+.[0-9]+)', str(x)))
dataS['price'] = dataS['priceNew'].apply(lambda x: float(x[0]))
dataS = dataS.drop('priceNew', axis=1)
dataS.info()

***סעיף 5***

In [ ]:
dataS['event_type'].unique()

In [ ]:
ax = dataS['event_type'].value_counts().plot(kind='bar',
                                    figsize=(8,5), color = 'purple',
                                    title="How many of each type")
ax.set_xlabel("event_type")
ax.set_ylabel("Frequency")
for i, v in enumerate(dataS['event_type'].value_counts()):
    ax.annotate(str(v), xy=(i, v), ha='center', va='bottom')
    
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)

plt.show()

**סעיף 6**

In [ ]:
#columns a+b+c+d:
session_data = dataS.groupby('user_session').last()[['user_id', 'funnel_number', 'index_in_funnel']].reset_index()
session_data

In [ ]:
#columns e+f:
session_data = session_data.merge(dataS.groupby('user_session')['event_type'].size().reset_index(name='amount_of_events'), on='user_session')
session_data = session_data.merge(dataS.groupby('user_session')['duration_to_next_event'].sum().reset_index(name='session_duration'), on='user_session')
session_data

In [ ]:
#column g:
view_events = dataS.iloc[np.where(dataS['event_type'] == 'view')][['user_session','product_id']]
session_data = session_data.merge(view_events.groupby('user_session')['product_id'].agg(lambda x: x.tolist()).reset_index(name='list_of_viewed'), on='user_session', how='outer')
session_data

In [ ]:
#column h:
cart_events = dataS.iloc[np.where(dataS['event_type'] == 'cart')][['user_session','product_id']]
session_data = session_data.merge(cart_events.groupby('user_session')['product_id'].agg(lambda x: x.tolist()).reset_index(name='list_of_added_to_cart'), on='user_session', how='outer')
session_data

In [ ]:
#column i:
purchase_events = dataS.iloc[np.where(dataS['event_type'] == 'purchase')][['user_session','product_id']]
session_data = session_data.merge(purchase_events.groupby('user_session')['product_id'].agg(lambda x: x.tolist()).reset_index(name='list_of_purchased'), on='user_session', how='outer')

In [ ]:
session_data